Noms et prénoms des membres du groupes:

-

# Vision industrielle - TP noté

### Exécuter la cellule suivante afin de charger l'ensemble des outils et fonctions nécessaires

In [ ]:
from IPython.core.display import HTML
from functools import partial
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib notebook
#%matplotlib inline
import pip
pip.main('install imageio'.split())
import imageio
import skimage.morphology
import skimage.segmentation
import skimage.feature
import skimage.data


plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['image.interpolation'] = 'none'
plt.rcParams['figure.max_open_warning'] = 100
plt.rcParams['figure.figsize'] = 2, 2

def format_coord(x, y, image):
    x = int(x + 0.5)
    y = int(y + 0.5)
    try:
        val = str(image[y, x])
    except IndexError:
        val = "?"
    return "color={} @ [{}, {}]".format(val, y, x)

def display_image_nb(image, title='', cmap=None):
    fig, ax = plt.subplots(1,1)
    if cmap is None:
        ax.matshow(image)
    else:
        ax.matshow(image, cmap=cmap)
    ax.set_xlabel(title)
    ax.format_coord = partial(format_coord, image=image)
    #return fig, ax

def display_image_color(image, title=''):
    fig, ax = plt.subplots(1,1)
    ax.imshow(image)
    ax.set_xlabel(title)
    ax.format_coord = partial(format_coord, image=image)
    #return fig, ax

def draw_rectangle(y1, x1, y2, x2, color='red', ax=None, alpha=1):
    """ Add a rectangle on an image """
    if ax is None:
        ax = plt.gca()
    rect = mpatches.Rectangle((x1, y1), x2 - x1, y2 - y1,
                              fill=False, edgecolor=color, linewidth=2, alpha=alpha)
    ax.add_patch(rect)

print("OK let's go")


# Exercice 1: Trouver les différences

Compléter le code suivant afin de trouver les différences entre les deux images img_nb_1, img_nb_2


In [ ]:
# chargement des images
img_nb_1 = imageio.imread('http://srand.fr/data/spot_diff_1.png')
img_nb_2 = imageio.imread('http://srand.fr/data/spot_diff_2.png')
display_image_nb(img_nb_1, title='img1')
display_image_nb(img_nb_2, title='img2')


- Calculer la différence entre **img_nb_1** et **img_nb_2**:

In [ ]:
img_diff =

- Afficher l'image des différences

- Calculer le masque binaire correspondant aux positions où une différence a été détectée (le masque doit valoir **True** là où une différence a été détectée)
- Afficher ce masque

In [ ]:
mask = 

# afficher le masque : 



- Utiliser une opération morphologique afin de regrouper les régions du masque ayant des valeurs à True (i.e il s'agit ici d'enlever les trous dans les zones blanches)

In [ ]:
import skimage.morphology

new_mask = 

Afficher l'image original, et dessiner un rectange autour des zones dans lesquelles il y a eu une différence.

Aide: le code est quasiment similaire à celui du tp1 (cellule 31: http://srand.fr/cours/tp1_red_ball_detection-Correction_one_page.html )

- Combien de différences présentent les deux images

# Exercice 2: Détection de véhicules

Dans cet exercice vous utiliserez des méthodes de machine learning afin de détecter la présence de véhicules dans des images.

Exemple de détection obtenu en faisant cet exercice: 
![alt text](http://srand.fr/data/car_detected.png "Car detected")

## Téléchargement des données pour l'apprentissage

Télécharger l'archive suivantes: http://srand.fr/data/car.tar.gz puis décompressez la. (vous pouvez utilisez le code de la cellule suivante ou un outil tel que winrar).

In [ ]:
import tarfile
tar_obj = tarfile.open("A_REMPLACER")   # <-- mettre le chemin vers l'archive (astuce: touche tabulation pour la complétion)
tar_obj.extractall('./')

Le fichier README.txt dans le répertoire CarData décrit le contenu de l'archive: 

[...]

1. Training images
These are located in CarData/TrainImages/. The positive (car) images are named pos-n.pgm (0 <= n <= 549). The negative (non-car) images are named similarly as neg-n.pgm (0 <= n <= 499).
The training images are all 100x40 pixels in size.

[...]


Aller dans le répertoire CarData/TrainImages et visualiser quelques-unes des images. 
Nous utiliserons l'ensemble de ces images pour l'apprentissage.

- Le code suivant permet d'ouvrir les différentes images (positives et négatives) et de les stocker dans des listes (images_positives et images_negatives)

In [ ]:
import glob
path = './CarData/TrainImages/'
negative_files = glob.glob(path + 'neg-*.pgm')
positive_files = glob.glob(path + 'pos-*.pgm')
images_positives = [imageio.imread(positive_file) for positive_file in positive_files]
images_negatives = [imageio.imread(negative_file) for negative_file in negative_files]

In [ ]:
# Affichage de la première image positive
display_image_nb(images_positives[0])

In [ ]:
# Affichage de la 5 ème image negative
display_image_nb(images_negatives[4])

- À quoi correspondent les images "positives" ? 
- À quoi correspondent les images "negatives" ? 
- Combien d'images avons nous dans chaque classe ?
- Quelle est la taille de chaque image ?

## Apprentissage

De façon à pré-traité les images, et à extraire un vecteur de caractéristiques décrivant le contenu de chaque image nous utiliserons la fonction suivante. Elle calcule des "local_binary_pattern" pour chaque image (comme vu lors du tp 3).

In [ ]:
from skimage.feature import local_binary_pattern

def compute_feature_vect(image, debug=False):
    radius = 2
    n_points = 8 * radius
    METHOD = 'uniform'
    lbp = local_binary_pattern(image, n_points, radius, METHOD)
    n_bins = 18
    hists = []
    if debug:
        display_image_nb(image)
    I = image.shape[0] / 6
    J = image.shape[1] / 5
    step = 10
    #tab = skimage.util.view_as_blocks(lbp, (I, J))
    tab = skimage.util.view_as_windows(lbp, (I,J), step=step)
    c = 0
    colors = ['blue', 'green', 'red']
    for i in range(tab.shape[0]):
        for j in range(tab.shape[1]):
            c+=1
            sub = tab[i, j]
            hist, _ = np.histogram(sub, normed=True, bins=n_bins, range=(0, n_bins))
            hists.append(hist.flatten())
            if debug:
                draw_rectangle(i*step, j*step, i*step+I-1, j*step+J-1, color=colors[c%len(colors)], alpha=0.8)
    return np.array(hists).flatten()

Afin de pouvoir utiliser les outils d'apprentissages automatisés, les données doivent être mise au format suivant:

    - X = data  (un vecteur avec tous les images pré-traités)
    - Y = label (un vecteur avec les labels associés à chaque image)

Nous utiliserons le label **"negative"** pour les images de la classe negative, et le label **"positive"** pour les images de la classe positive.


**Compléter le code suivant:**

In [ ]:
X = []
Y = []

for image in images_negatives:
    X.append(compute_feature_vect(image))
    Y.append(          )
    
for image in images_positives:
    X.append(compute_feature_vect(image))
    Y.append(         )
    

In [ ]:
print(Y)

Entrainer un classifieur de type K-nearest neighbors (en utilisant 10 voisins).
A l'aide de la fonction cross_val_score, calculer les pourcentages d'accuracy obtenus sur le jeu d'entrainement.

- Que pouvez vous dire sur ces pourcentages ? 
- Quel serait l'accuracy d'un classifieur aléatoire sur ce jeu de donnée ? (Utiliser un DummyClassifier pour vérifier)



In [ ]:
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import cross_val_score



clf_k_nearest_neighbors =  
scores_cross_validation = 

print(scores_cross_validation)

In [ ]:
from sklearn.dummy import DummyClassifier
classifieur_tout_pourri = 
scores_cross_validation = 
print(scores_cross_validation)


Entrainer un classifieur de type AdaBoost (en utilisant 200 estimateurs (n_estimators=200)).
A l'aide de la fonction cross_val_score, calculer les pourcentages d'accuracy obtenus avec ce classifieur sur le jeu d'entrainement.

Comparer ces résultats aux résultats obtenus avec le classifieur de type K-nearest neighbors.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf_adaboost = AdaBoostClassifier(                 )
scores_cross_validation = 
print(scores_cross_validation)


Dans la suite nous utiliserons le classifieur *clf_adaboost*.

Lancer l'apprentissage pour ce classifieur (méthode fit)

In [ ]:
clf_adaboost.




# Reconnaissance d'un véhicule

Nous diposons à présent d'un classifieur de type AdaBoost qui sait prédire la présence d'un véhicule dans une image de 40x100 pixels.

Tester ce classifieur sur la première image posivite, et sur la 5 ème image negative.

** N'oubliez pas d'utiliser la méthode compute_feature_vect afin de prétraiter les images **

In [ ]:
image_test1 = images_positives[0]
display_image_nb(image_test1)
prediction_image_test_1 = clf_adaboost.     # a completer        
print(prediction_image_test_1)


image_test2 = images_negatives[4]
display_image_nb(image_test2)
prediction_image_test_2 = clf_adaboost.     # a completer        
print(prediction_image_test_2)


# Détection d'un véhicule dans une image

Nous nous interessons ici à la détection d'une véhicule dans une image plus grande que 40x100 pixels. Afin de détecter un véhicule nous utiliserons une fenêtre glissante comme vu lors du tp3. La fonction suivante vous permet d'utiliser une fenêtre glissante.

In [ ]:
def check_image(image_test, window_size = (40, 100), clf=None, compute_feat_function=None):
    """
    image_test: image en entree
    window_size: taille de la fenetre glissante (correspond a la taille des images dans l'ensemble d'apprentissage)
    clf: votre classifieur deja entrainé
    compute_feature_vect: la fonction de pré-traitement des images
    """
    STEP = max(window_size) /5
    tab = skimage.util.view_as_windows(image_test, window_size, STEP)
    display_image_nb(image_test)
    for i in range(tab.shape[0]):
        for j in range(tab.shape[1]):
            block = tab[i, j]
            if compute_feat_function is not None:
                block = compute_feat_function(block)
            prediction = clf.predict(block.flatten())
            #print(prediction)
            if prediction == 'positive':
                draw_rectangle(i*STEP, j*STEP, i*STEP+window_size[0], j*STEP+window_size[1], color='green')


In [ ]:
# Exemple d'utilisation
im1 = imageio.imread('http://srand.fr/data/test-43.pgm')
display_image_nb(im1)
check_image(im1, window_size=(40, 100), clf=clf_adaboost, compute_feat_function=compute_feature_vect)

Testez le classifieur à l'aide de la méthode check_image sur 10 images du dossier CarData/TestImages/  (afficher les 10 résultats)

Vous pouvez utiliser la méthode glob.glob ainsi qu'une boucle, ou utiliser le chemin complet vers les images.

# Fin

Merci d'envoyer votre tp à Laurent.f.george@ gmail. com.

(File->print_preview, puis enregistrer_sous).